In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np 
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from keras.models import load_model


In [2]:
### read the model file 
model=load_model('my_model.h5')
## open the pickle fiels
with open('label_gender.pkl','rb') as file:
    label_gender=pickle.load(file)
    
with open('onehot_code_geo.pkl','rb') as file:
    onehot_code_geo=pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)


In [3]:
# Example input data
input_data = {
    'CreditScore': 800,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 30,
    'Tenure': 3,
    'Balance': 80000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}


In [4]:
# One-hot encode 'Geography'
geo_encode=onehot_code_geo.transform([[input_data['Geography']]]).toarray()
geo_encode_df=pd.DataFrame(geo_encode,columns=onehot_code_geo.get_feature_names_out(['Geography']))
geo_encode_df


e:\RNN Project\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df=pd.DataFrame([input_data])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,France,Male,30,3,80000,2,1,1,50000


In [6]:
input_df['Gender']=label_gender.transform(input_df['Gender'])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,France,1,30,3,80000,2,1,1,50000


In [7]:
## data conncet 
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encode_df],axis=1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800,1,30,3,80000,2,1,1,50000,1.0,0.0,0.0


In [8]:
###scaling the data
input_scal=scaler.transform(input_df)
input_scal


array([[ 1.55126923,  0.91324755, -0.84724032, -0.69250056,  0.06092123,
         0.81066646,  0.64609167,  0.97067965, -0.87139988,  1.00626965,
        -0.57985213, -0.57965968]])

In [9]:
##prediction the data
prediction=model.predict(input_scal)
prediction


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


array([[0.0153914]], dtype=float32)

In [10]:
prediction_proba = prediction[0][0]
prediction_proba


0.015391399

In [11]:
if prediction_proba >0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')


The customer is not likely to churn.
